In [15]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
from pathlib import Path
from llama_index import download_loader

MarkdownReader = download_loader("MarkdownReader")

loader = MarkdownReader()
resume_documents = loader.load_data(file=Path("../data/resume.md"), extra_info={'file_name': 'resume.md'})
job_description_documents = loader.load_data(file=Path("../data/job_description.md"), extra_info={'file_name': 'job_description.md'})

In [8]:
resume_documents

[Document(id_='22ead098-80ea-4916-b916-9133bd61c06b', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='acdbad31ef53ae861858de967cd3cd18569a631d26c9aa2d948a92e6c56d290f', text='\n**Contact Information**\n\n- 123 Main Street, Anytown, USA\n- Phone: (123) 456-7890\n- Email: joedoe@example.com\n\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='920ea733-cc71-4c92-8231-f299d0d33a93', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='59b37a1c8350277a9cd7c26bcf89bbcd578da30014d04d96012751f6d40df21e', text='\nInnovative software engineer with a proven ability to develop high-performance applications and technical innovations. Seeking a challenging role to utilize my skills and knowledge, enhance my e

In [9]:
job_description_documents

[Document(id_='7a7b3fb5-163c-4aa0-a5ea-69f0eafcfd96', embedding=None, metadata={'file_name': 'job_description.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='81a5d8004ec41a9c7f0fb57759532a0d3c37e47637d7bc78dd3470bc5b542a69', text='\n**Location**: Anytown, USA\n\n**Company**: FutureTech Inc.\n\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='af2f3333-a92e-4f29-855c-06855aa62734', embedding=None, metadata={'file_name': 'job_description.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9c8e0a4e2e67453bcb94becdc77a695cc7becbdedbf818c99e8862b82f0a1682', text='\nFutureTech Inc. is a leading technology company that specializes in creating innovative software solutions to address everyday challenges. We are seeking a talented and motivated Software Engineer to join our dynamic team.\n\n', s

In [10]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

resume_nodes = parser.get_nodes_from_documents(resume_documents)
job_description_nodes = parser.get_nodes_from_documents(job_description_documents)

In [11]:
resume_nodes

[TextNode(id_='86692732-2841-43a3-b978-663d2f73b061', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='22ead098-80ea-4916-b916-9133bd61c06b', node_type=None, metadata={'file_name': 'resume.md'}, hash='acdbad31ef53ae861858de967cd3cd18569a631d26c9aa2d948a92e6c56d290f')}, hash='e2c34d72fd18412cf53f844ddf11a0c1b89cb919d9f4bea7fc1155482dfa822c', text='**Contact Information**\n\n- 123 Main Street, Anytown, USA\n- Phone: (123) 456-7890\n- Email: joedoe@example.com', start_char_idx=0, end_char_idx=108, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='acf5f033-c66f-4ca8-9148-d51e54adc7f1', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='920ea733-c

In [16]:
from llama_index.schema import NodeRelationship
from llama_index import VectorStoreIndex

resume_nodes[-1].relationships[NodeRelationship.NEXT] = job_description_nodes[0].node_id
job_description_nodes[0].relationships[NodeRelationship.PREVIOUS] = resume_nodes[-1].node_id
nodes = resume_nodes + job_description_nodes

index = VectorStoreIndex(nodes)

In [17]:
from llama_index import StorageContext, ListIndex

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index1 = VectorStoreIndex(nodes, storage_context=storage_context)
index2 = ListIndex(nodes, storage_context=storage_context)

In [18]:
r = index1.as_retriever().retrieve("I am a software engineer")
r

[NodeWithScore(node=TextNode(id_='986c2854-ef4a-4b66-9560-e58cad35c049', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e6d615ff-2ea0-4a46-8f00-86011aa0f9f3', node_type=None, metadata={'file_name': 'resume.md'}, hash='356615b4894df7c939eb44ae393199faae7289428b4c86284012c21a9f5b12b6')}, hash='b9739ea60606d5e2b594e3a009b9c6566069c2cff0dd01a986e398156cd668b0', text='**Software Engineer, ABC Company, Anytown, USA (2019 - present)**\n\n- Designed and implemented software applications in Java and Python.\n- Collaborated with cross-functional teams to define, design, and ship new features.\n- Continuously discovered, evaluated, and implemented new technologies to maximize development efficiency.', start_char_idx=0, end_char_idx=327, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8369189

In [20]:
s = r[0]
s

NodeWithScore(node=TextNode(id_='986c2854-ef4a-4b66-9560-e58cad35c049', embedding=None, metadata={'file_name': 'resume.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e6d615ff-2ea0-4a46-8f00-86011aa0f9f3', node_type=None, metadata={'file_name': 'resume.md'}, hash='356615b4894df7c939eb44ae393199faae7289428b4c86284012c21a9f5b12b6')}, hash='b9739ea60606d5e2b594e3a009b9c6566069c2cff0dd01a986e398156cd668b0', text='**Software Engineer, ABC Company, Anytown, USA (2019 - present)**\n\n- Designed and implemented software applications in Java and Python.\n- Collaborated with cross-functional teams to define, design, and ship new features.\n- Continuously discovered, evaluated, and implemented new technologies to maximize development efficiency.', start_char_idx=0, end_char_idx=327, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.83691898

In [23]:
from pprint import pprint
pprint(s.json())

('{"node": {"id_": "986c2854-ef4a-4b66-9560-e58cad35c049", "embedding": null, '
 '"metadata": {"file_name": "resume.md"}, "excluded_embed_metadata_keys": [], '
 '"excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": '
 '"e6d615ff-2ea0-4a46-8f00-86011aa0f9f3", "node_type": null, "metadata": '
 '{"file_name": "resume.md"}, "hash": '
 '"356615b4894df7c939eb44ae393199faae7289428b4c86284012c21a9f5b12b6"}}, '
 '"hash": "b9739ea60606d5e2b594e3a009b9c6566069c2cff0dd01a986e398156cd668b0", '
 '"text": "**Software Engineer, ABC Company, Anytown, USA (2019 - '
 'present)**\\n\\n- Designed and implemented software applications in Java and '
 'Python.\\n- Collaborated with cross-functional teams to define, design, and '
 'ship new features.\\n- Continuously discovered, evaluated, and implemented '
 'new technologies to maximize development efficiency.", "start_char_idx": 0, '
 '"end_char_idx": 327, "text_template": "{metadata_str}\\n\\n{content}", '
 '"metadata_template": "{key}: {val

In [24]:
s.node.text

'**Software Engineer, ABC Company, Anytown, USA (2019 - present)**\n\n- Designed and implemented software applications in Java and Python.\n- Collaborated with cross-functional teams to define, design, and ship new features.\n- Continuously discovered, evaluated, and implemented new technologies to maximize development efficiency.'